## BOW -> Classification

This is a baseline where we convert sentences into a BOW matrix and then classify for type of bias in the entire sentence. The main difference to other weak labeling functions is that in this case we feed the entire input into the classifier with no knowledge of what the actual biased word is .

In [ ]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
params = Params.read_params("experiment_params.json")

In [5]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params.dataset)

03/24/2020 09:39:49 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
386it [00:00, 4822.63it/s]


In [6]:
from src.utils.baseline_utils import get_bow_matrix

In [7]:
bow_features = get_bow_matrix(params.dataset, dataset)

03/24/2020 09:39:50 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [8]:
dataset.add_data(bow_features, "bow_features")

### Start of Classification

In [9]:
from src.utils.classification_utils import run_bootstrapping

In [10]:
num_vocab = bow_features.shape[1]
params.final_task['input_dim'] = num_vocab
params.final_task['hidden_dim'] = num_vocab//2

In [14]:
bert_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [15]:
statistics = run_bootstrapping(bert_experiment, dataset, params.final_task, num_bootstrap_iters=5, input_key='bow_features', label_key='bias_label', threshold=0.42)

In [16]:
statistics

{'auc': [(0.5918036929057339, 0.7254975704567541), 0.6661101392938128],
 'accuracy': [(0.6244444444444445, 0.7132653061224489), 0.6676190476190476]}

In [ ]:
# Can also run the following to get the statistics for each epoch - without any bootstrapping
# we can average over each of the epochs; and then get the max 
bert_dataloader = dataset.return_dataloader(
    batch_size=params.final_task['training_params']['batch_size']
)

_, stats = bert_experiment.train_model(
    bert_dataloader, 
    bert_dataloader,
    input_key="bow_features",
    label_key="bias_label",
    threshold=0.42,
    print_loss_every=0 #set to anything >0 to actually print 
)